# IT Service Ticket Classification

Este notebook implementa um sistema de classificação de tickets de suporte de TI utilizando RAG (Retrieval Augmented Generation) com LangGraph.

**Entrada:** texto do ticket (string)  
**Saída:** `{"classe": "...", "justificativa": "..."}`

## 1. Carregamento e Preparação dos Dados

In [ ]:
from classifier.data import load_dataset, train_test_split_stratified

In [3]:
# Carregar dataset
df, classes = load_dataset()
print(f"Total de tickets: {len(df):,}")
df.head()

Total de tickets: 47,837


,Document,Topic_group
0,connection with icon icon dear please setup ic...,Hardware
1,work experience user work experience user hi w...,Access
2,requesting for meeting requesting meeting hi p...,Hardware
3,reset passwords for external accounts re expir...,Access
4,mail verification warning hi has got attached ...,Miscellaneous


In [7]:
# Classes obtidas do dataset
print(f"Classes:\n- {'\n- '.join(classes)}")

Classes:
- Access
- Administrative rights
- HR Support
- Hardware
- Internal Project
- Miscellaneous
- Purchase
- Storage


In [10]:
# Split estratificado: treino para RAG, teste para avaliação (200 tickets)
train_df, test_df = train_test_split_stratified(df, test_size=200)

print(f"Treino: {len(train_df):,} tickets")
print(f"Teste:  {len(test_df)} tickets")

Treino: 47,637 tickets
Teste:  200 tickets
